In [1]:
#load dataset
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

df = pd.read_csv('/content/drive/MyDrive/May22-Galileo/cmm_erdos_2022/pharmacy_tx.csv')

In [5]:
# Group pcn values by "group"
# if pcn is empty, select all entries with same group 
# and give a new pcn_value
# else pcn value exists

i = 0
for key, val in df.groupby(['group'])['pcn'].agg(pd.Series.mode).items():
    if len(val) <= 0:
        df.loc[df['group'] == key, 'pcn'] = 'pcn_' + str(i)
        i += 1

In [6]:
# Same explanation holds above
j = 0
for key, val in df.groupby(['pcn'])['group'].agg(pd.Series.mode).items():
    if len(val) <= 0:
        df.loc[df['pcn'] == key, 'group'] = 'group_' + str(j)
        j += 1

In [7]:
# The remaining entries are does with Nan values for both pcn and group
# Just fill with the next pcn and group values respectively


df['pcn'] = df['pcn'].fillna('pcn_' + str(i))
df['group'] = df['group'].fillna('group_' + str(j))

In [8]:
df[['pcn', 'group']].isnull().any()

pcn      False
group    False
dtype: bool

In [9]:
#Get rid of unknown values
col_var = 'pcn'
df[col_var] = df[col_var].fillna('Unknown')

col_var = 'group'
df[col_var] = df[col_var].fillna('Unknown')

col_var = 'drug'
df[col_var] = df[col_var].fillna('Unknown')


In [10]:
#Combine diagnosises that do not have a lot of values into others
labels = np.where(df.diagnosis.value_counts() < 100)
change_labels = df.diagnosis.value_counts().index[labels[0]]
df['diagnosis'] = ['Others' if val in change_labels else val for val in df.diagnosis]

In [11]:
#Create column for branded or generic
col_var = 'drug'
df['drug_group'] = [val.split(' ')[0] for val in df[col_var]]
df['drug_type']= [val.split(' ')[1] for val in df[col_var]]

In [12]:
def ends_with(dat):
  return dat[-3:]

In [13]:
df['drug_class'] = df['drug_type'].apply(ends_with)

In [14]:
del df['tx_date']
del df['pharmacy']
del df['group']

In [16]:
df.to_csv("pharmacyClean.csv")